# WHIRL in ProbLog

Februari 2015  
[Wannes Meert](mailto:wannes.meert@cs.kuleuven.be), [Anton Dries](mailto:anton.dries@cs.kuleuven.be), DTAI Research Group, KU Leuven

(Cohen 2000b):

> WHIRL is an “information representation language” that synergistically combines properties of logic-based and text-based representation systems. WHIRL is a subset of Datalog that has been extended by introducing an atomic type for textual entities, an atomic operation for computing textual similarity, and a “soft” semantics; that is, inferences in WHIRL are associated with numeric scores, and presented to the user in decreasing order by score. 

(Cohen, 2000a) W. W. Cohen. Whirl: A word-based information representation language. Artificial Intelligence, 118(1):163–196, 2000.

(Cohen, 2000b) W. W. Cohen. Data integration using similarity joins and a word-based information representation language. ACM Transactions on Information Systems (TOIS), 18(3):288–321, 2000.

## Differences between WHIRL and ProbLog

### Recursion and negation

WHIRL does not allow for recursion or negation. Additionally, only one `many/2` predicate is allowed in a query. These restrictions do not apply to Problog because it knows how to handle such situations (e.g. the inclusion-exclusion principle).

### Search strategy

Problog does not use a find-best-substitution approach like the WHIRL system that uses an A* search strategy. The default reasoning technique in Problog finds all proofs. But the optimizations present in WHIRL can be implemented in the Python part.

### Storage

A more efficient way of storing all the reviews is to preprocess all reviews and use compact vector representations in ProbLog. This is also how it is represented in Cohen (200a). For illustratieve purposes, in this example we use full strings in ProbLog.

## Data

In this example, we will be using movie reviews, listings and other information. The data is represented using Prolog facts:

- `listing/3`: Movie theater listing
- `review/2`: Movie review
- `academy_award/1`: Category of oscars
- `winner/3`: Which movie won wat category in what year

In [16]:
!head -n 49 whirl.pl | tail -n 14

listing('Roberts Theater Chatham', 'Brassed Off', '7:15 - 9:10').
listing('Berkeley Cinema', 'Hercules', '2:00 - 4:15 - 7:30').
listing('Sony Mountainside Theater', 'Men in Black', '7:40 - 8:40 - 9:30 - 10:10').

review('Men in Black, 1997', '(***) One of the summer s biggest hits, this ... a comedy about space aliens with Will Smith ...').
review('Face/Off, 1997',     '(**1/2) After a somewhat slow start, Cage and Travolta').
review('Space Balls, 1987',  '(*1/2) While not one of Mel Brooks better efforts, this Star Wars spoof ... a comedy about space').
review('Hercules',  'Animated Disney film').
review('The Lord of the Rings: The Fellowship of the Ring, 2001', 'An epic fantasy film directed by Peter Jackson based on the first volume of J. R. R. Tolkien The Lord of the Rings. It is the first installment in the The Lord of the Rings film series, and was followed by The Two Towers (2002) and The Return of the King (2003), based on the second and third volumes of The Lord of the Rings.'

## Similarity Metric and Score

Similarity is expressed as $X \sim Y$ in WHIRL and `similar(X,Y)` in our version. It has a score value associated it with it that in ProbLog will be expressed as a probability associated with a probabilistic fact.

The $SCORE(B,\theta)$ of an expression $B$ and substitution $\theta$ is computed using this similarity.

- if $B$ is a literal and a fact in the DB: $SCORE(B,\theta) = s$ (the score, probability or weight of the fact)
- if $B$ is a literal but not in the DB: $SCORE(B,\theta) = 0$
- if $B$ is a similarity literal $X \sim Y$: $SCORE(B,\theta) = SIM(\vec{x}, \vec{y})$, where $\vec{x} = X\theta$ and $\vec{y} = Y\theta$ and $\theta$ a ground substitution.

The similarity ($SIM$, $X \sim Y$ or `similar(X,Y)`) between sentences and documents is computed using the TF-IDF metric (see `whirl.py` for the complete code).

We use NLTK to translate a sentence into a list of stems:

In [5]:
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(text):
  return (stemmer.stem(token) for token in word_tokenize(text))

And Scikit-learn to compute the TF-IDF parameters and the similarity between sentences:

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

texts = ['Text topic1', 'And text topic2'] # In our setting, all the movie reviews are loaded
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
weights = tfidf.fit_transform(texts)

def similarity(sentence1, sentence2):
  return cosine_similarity(tfidf.transform(sentence1),tfidf.transform(sentence2))

print(tfidf.get_feature_names())
weights.toarray()

['text', 'topic1', 'topic2']


array([[ 0.57973867,  0.81480247,  0.        ],
       [ 0.57973867,  0.        ,  0.81480247]])

## Run Python within ProbLog

Python code can be run from ProbLog by including a Python file using `load_external/1` and calling a Python method using `call_external/2`.

In [3]:
%%script problog-cli.py
:- load_external('whirl.py').

P::similar(X,Y) :-
    call_external(similarity(X,Y), P),
    P > 0.0.
            
query(similar('comedy Smith', 'space Smith')).
query(similar('comedy Smith', 'space Brooks')).
query(similar('comedy Smith', 'Will Smith')).

	  similar('comedy Smith','Will Smith') : 0.77828292
	similar('comedy Smith','space Brooks') : 0
	 similar('comedy Smith','space Smith') : 0.60572431


The `fail/0` in the `similar/2` call is to instruct ProbLog not to keep track of two entities that are completely unrelated (instead of performing computations with probability 0.0).

The results can be interpreted as follows:

- 'comedy Smith' and 'Will Smith' has the highest probability because there is only one movie that is a comedy with Will Smith
- 'comedy Smith' and 'space Smith' is a bit lower because there are two space comedies
- 'comedy Smith' and 'space Brooks' is zero because there is no overlap

### Optimizations

We can adapt the `similar/2` predicate to use a more efficient search strategy and only unify if the two terms are very similar. If `similar\2` fails, the atom is ignored and the combination of the two terms is not further explored.

In [6]:
%%script problog-cli.py
:- load_external('whirl.py').

P::similar(X,Y) :-
    call_external(similarity(X,Y), P),
    P > 0.7. % Only keep likely matches
            
query(similar('comedy Smith', 'space Smith')).
query(similar('comedy Smith', 'space Brooks')).
query(similar('comedy Smith', 'Will Smith')).

	  similar('comedy Smith','Will Smith') : 0.77828292
	similar('comedy Smith','space Brooks') : 0
	 similar('comedy Smith','space Smith') : 0


An optimisation proposed in Cohen (2000a) that is possible in the Python part is to build an (inverted) index of similar documents instead of trying all combinations:

> Consider x􏲄, corresponding to the document "Armadillos, Inc". Due to the frequent term "Inc", there will be many documents Y that have nonzero similarity to x􏲄, and it will be expensive to retrieve all of these documents Y and compute their similarity to x􏲄. One way of avoiding this expense is to start by retrieving a small number of documents Y that are likely to be highly similar to x􏲄. In this case, one might use an index to find all Y's that contain the rare term "Armadillos". Since "Armadillos" is rare, this step will be inexpensive, and the Y's retrieved in this step must be somewhat similar to x􏲄.

## Conjunctive Queries

The score of a conjunctive query $B_1 \lor \dots \lor B_k$ is

\begin{equation*}
SCORE(B_1 \land \dots \land B_k, \theta) = 1 - \prod_{i=1}^{k}SCORE(B_i,\theta)
\end{equation*}

with $\theta$ the ground unification.

### Q1

Find reviews about comedies with space aliens

In [5]:
%%script --out result problog-cli.py
:- consult(whirl).

q1(Movie, Review) :- review(Movie, Review),
                     similar(Review, 'comedy with space aliens').
query(q1(X,Y)).

In [6]:
problogResult(result)

### Q2

Soft database join to see times and reviews

In [9]:
%%script --out result problog-cli.py
:- consult(whirl).

q2(Cinema,Movie1,Times,Review) :- listing(Cinema, Movie1, Times),
                                  review(Movie2, Review),
                                  similar(Movie1,Movie2).
query(q2(C,M,T,R)).

In [10]:
problogResult(result)

### Q3

Database join to see times and reviews. Whill be empty if no exactly matching movie titles

In [38]:
%%script problog-cli.py
:- consult(whirl).

q3(Cinema,Movie,Times,Review) :- listing(Cinema,Movie,Times),
                                 review(Movie,Review).
query(q3(C,M,T,R)).

	q3('Berkeley Cinema','Hercules','2:00 - 4:15 - 7:30','Animated Disney film') : 1


### Q4

See where the latest science fiction comedy is playing.

In [42]:
%%script problog-cli.py
:- consult(whirl).

q4(Movie1) :- listing(Cinema, Movie1, Times), review(Movie2, Review),
              similar(Movie1, Movie2),
              similar(Review,"comedy with space aliens").
query(q4(M)).

	 q4('Brassed Off') : 0
	    q4('Hercules') : 0
	q4('Men in Black') : 0.46729939


## Disjunctive Queries

The probability of a disjunctive query is

\begin{equation*}
SCORE(a, \theta) = 1 - \prod_{<\theta,Body_i> \in SUPPORT(a)}(1-SCORE(Body_i,\theta))
\end{equation*}

with SUPPORT for literal $a = v(a_1,\dots,a_k)$ is the set of pairs $<\theta,Body_i>$ for which

- $v(X_1,\dots,X_k)$ :- $Body_i.$ is a rule
- $\exists \theta: v(X_1,\dots,X_k)\theta = a$
- $Body_i\theta$ is ground, and $SCORE(Body_i,\theta) > 0$

### Q4 (disjunctive)

Disjunctive version of Q4. Finds cinemas that are playing either a science fiction comedy or an animated film produced by Disney

In [45]:
%%script problog-cli.py
:- consult(whirl).

view1(Cinema) :- listing(Cinema, Movie1, Times), 
                 review(Movie2, Review),
                 similar(Movie1, Movie2),
                 similar(Review, "comedy with space aliens").
view1(Cinema) :- listing(Cinema, Movie1, Times),
                 review(Movie2, Review),
                 similar(Movie1, Movie2),
                 similar(Review, "animated Walt Disney film").
q4a(Cinema) :- view1(Cinema).
query(q4a(C)).

	          q4a('Berkeley Cinema') : 1
	  q4a('Roberts Theater Chatham') : 0
	q4a('Sony Mountainside Theater') : 0.46729939


## Soft Universal Quantification

Soft universal quantification is expressed as `many(Template, Test)` and defined as the weighted average of the scores where the test succeeds for the given template:

\begin{equation*}
SCORE(many(p(Y_1, \dots, Y_k), Test, \theta) = \sum_{<s,a_1,\dots,a_k> \in P} \frac{s}{S}\cdot SCORE\left(Test, (\theta \cup {Y_1=a_1, \dots, Y_k=a_k})\right)
\end{equation*}

with $P$ the set of all tuples $<s,a_1,\dots,a_k>$ such that $p(a_1,\dots,a_k)$ in the DB with score $s$. and $S = \sum_{<s,a_1,\dots,a_k>\in P}s$.

### Q5 (deterministic)

Movie with Will Smith that won an award without fuzzy matching will find no results.

In [46]:
%%script problog-cli.py
:- consult(whirl).

q5a(Movie, Cat) :- review(Movie ,Review),
              similar(Review, "Will Smith"),
              academy_award(Cat), winner(Movie2, Cat),
              similar(Movie, Movie2).
query(q5a(M,C)).

Error during grounding: No clauses found for 'winner/2' at position 5:35.


### Q5

Movie that is currently playing and has *many* academy awards.

In [2]:
%%script problog-cli.py
:- consult(whirl).

q5(M) :-
    listing(_,M,_),
    % many(Template, Test) as findall(Test, Template, L)
    findall(winner(M,C,Y), (academy_award(C),winner(_,C,Y)), L),
    many_int_prob(L).
query(q5(M)).

	 q5('Brassed Off') : 0
	    q5('Hercules') : 0
	q5('Men in Black') : 0.5


The `many` predicate present in WHIRL can be implemented in ProbLog using `findall` and a custom predicate to count the weighted average:

In [50]:
%%script problog-cli.py

many_int([], P, N, 0) :- T is P+N, T = 0.
many_int([], P, N, S) :- T is P+N, T > 0, S is P/T.
many_int([H|T], PA, NA, S) :-
    (  call(H), PAN is PA + 1, NAN is NA;      % Test: true
     \+call(H), PAN is PA,     NAN is NA + 1), % Test: false
    many_int(T, PAN, NAN, S).

S::many_int_prob(L) :- many_int(L, 0, 0, S).

many(Template, Test) :-
    findall(Test, Template, L),
    many_int_prob(L).

## More information

https://dtai.cs.kuleuven.be/problog

*Document setup*: This document can be run as an IPython notebook. You can ignore the next code block.

In [4]:
class problogResult:
    def __init__(self,string):
        self.list = [s.rsplit(':',1) for s in string.split('\n') if s != '']
        for t in self.list: t[1] = float(t[1])
        self.list.sort(key=lambda x:x[1], reverse=True)
    def _repr_html_(self):
        return '<table>'+''.join(['<tr>{}'.format(''.join(['<td>{}'.format(cell) for cell in row])) for row in self.list])+'</table>'